In [14]:
import numpy as np
import networkx as nx

from itertools import combinations
from scipy.optimize import minimize

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import ParameterVector
from qiskit.primitives import BackendSamplerV2, BackendEstimatorV2
from qiskit_optimization.applications import Maxcut
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import SamplingVQE, QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer import AerSimulator

def qaoa_circuit(G, reps=1):
    r"""
    QAOA circuit for Max-cut \sum(I-ZZ). No longer follow qiskit QAOA circuit.
    If want to reproduce the circuit used in qiskit \sum(-ZZ), substitute
    gamma as -gamma will do.
    """
    gamma = ParameterVector('γ', reps)
    beta = ParameterVector('β', reps)

    q = QuantumRegister(len(G.nodes))
    qc = QuantumCircuit(q)

    qc.h(q)
    for p in range(reps):
        for u, v, d in G.edges(data=True):
            weight = d.get('weight', 1)
            qc.cx(u, v)
            qc.rz(-gamma[p]*weight, v)
            qc.cx(u, v)
        qc.rx(2*beta[p], q)
    return qc

In [15]:
n = 100
G = nx.fast_gnp_random_graph(n, 1., seed=0) # complete graph
max_cut = Maxcut(G)
qp = max_cut.to_quadratic_program()
# print(qp.prettyprint())

energy_op, offset = qp.to_ising()

# QAOA

In [17]:
reps = 10

initial_params = np.random.rand(2 * reps) * 2 * np.pi
ansatz = qaoa_circuit(G, reps=reps)

backend = AerSimulator(method='matrix_product_state')
sampler = BackendSamplerV2(backend=backend)
optimizer = 'COBYLA'

def obj_func(params):
    estimator = BackendEstimatorV2(backend=backend)
    qc = qaoa_circuit(G, reps)
    qc.measure_all()
    result = estimator.run([(qc, energy_op, params)]).result()[0]
    return result.data.evs

result = minimize(obj_func, initial_params, method=optimizer)
result

# VQE

## Linear entanglement

In [ ]:
backend = AerSimulator(method='matrix_product_state')
sampler = BackendSamplerV2(backend=backend)
# sampler = AerSamplerV2()

reps = 9
num_qubits = 100

qc = QuantumCircuit(num_qubits)
thetas = ParameterVector('theta', (reps + 1) * num_qubits)

# Initial rotation layer
for i in range(num_qubits):
    qc.ry(thetas[i], i)

for r in range(1, reps + 1):
    # Entangling layer
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)

    # Repeated rotation layer
    for i in range(num_qubits):
        qc.ry(thetas[r * num_qubits + i], i)

qc.measure_all()

initial_params = np.random.rand((reps + 1) * num_qubits) * 2 * np.pi
result = sampler.run([(qc, initial_params)]).result()[0]
result

## Full entanglement

In [ ]:
backend = AerSimulator(method='matrix_product_state')
sampler = BackendSamplerV2(backend=backend)
# sampler = AerSamplerV2()

reps = 5
num_qubits = 100

qc = QuantumCircuit(num_qubits)
thetas = ParameterVector('theta', (reps + 1) * num_qubits)
for i in range(num_qubits):
    qc.ry(thetas[i], i)

for r in range(1, reps + 1):
    for i, j in combinations(range(num_qubits), 2):
        qc.cx(i, j)

    for i in range(num_qubits):
        qc.ry(thetas[r * num_qubits + i], i)

qc.measure_all()

initial_params = np.random.rand((reps + 1) * num_qubits) * 2 * np.pi
result = sampler.run([(qc, initial_params)]).result()[0]
result